In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm

from statsmodels.regression.linear_model import OLS

In [2]:
df = pd.read_csv("DataWithoutPairs_USDRUB.csv")

In [3]:
def df_OLS_anal(df: pd.DataFrame):
    deltaA = np.array(df["AskAfter"] - df["AskBefore"])[1:-1]
    deltat = np.array(np.diff(df["Time"]))[1:]
    x = np.array(df["Volume"][1:-1])

    dA = np.array(deltaA / deltat)
    dx = np.array(x / deltat)
    ddx = np.array(np.diff(dx))
    ddA = np.array(np.diff(dA))

    OLSdf = pd.DataFrame({
        "SUM": ddA,
        "-rho": deltaA[:-1],
        "-rho kappa": x[1:],
        "rho (lambda + kappa)": x[:-1],
        "kappa + lambda": ddx
    })

    Regressand = OLSdf["SUM"]

    Regressors = OLSdf[["-rho", "-rho kappa", "rho (lambda + kappa)", "kappa + lambda"]]

    # Regressors = sm.add_constant(Regressors, prepend=False)

    model = OLS(Regressand, Regressors)

    res = model.fit()

    print(res.summary()) 

    rho = - res.params.iloc[0]
    kappa = - res.params.iloc[1] / rho
    lamb = res.params.iloc[2] / rho - kappa
    # lamb2 = res.params.iloc[3] - kappa
    

    print("rho = ", rho, "\nlambda = ", lamb, "\nkappa = ", kappa, "\n \n", end="")

In [ ]:
def df_OLS_anal_AO(df: pd.DataFrame):
    deltaA = np.array(df["AskAfter"] - df["AskBefore"])[1:-1]
    deltat = np.array(np.diff(df["Time"]))[1:]
    x = np.array(df["Volume"][1:-1])

    dA = np.array(deltaA / deltat)
    dx = np.array(x / deltat)
    ddx = np.array(np.diff(dx))
    ddA = np.array(np.diff(dA))

    OLSdf = pd.DataFrame({
        "SUM": ddA,
        "-rho": deltaA[:-1],
        "-rho kappa": x[1:],
        "rho (lambda + kappa)": x[:-1],
        "kappa + lambda": ddx
    })

    Regressand = OLSdf["SUM"]

    Regressors = OLSdf[["-rho", "-rho kappa", "rho (lambda + kappa)", "kappa + lambda"]]

    # Regressors = sm.add_constant(Regressors, prepend=False)

    model = OLS(Regressand, Regressors)

    res = model.fit()

    print(res.summary()) 

    rho = - res.params.iloc[0]
    kappa = - res.params.iloc[1] / rho
    lamb = res.params.iloc[2] / rho - kappa
    # lamb2 = res.params.iloc[3] - kappa
    

    print("rho = ", rho, "\nlambda = ", lamb, "\nkappa = ", kappa, "\n \n", end="")

In [7]:
df = pd.read_csv("DataWithoutPairs_USDRUB.csv")
# dfAsk = df[df["AskAfter"] - df["AskBefore"] > 0]
df = df[df["Volume"] > 1e5]
df_OLS_anal(df)

                                 OLS Regression Results                                
Dep. Variable:                    SUM   R-squared (uncentered):                   0.219
Model:                            OLS   Adj. R-squared (uncentered):              0.216
Method:                 Least Squares   F-statistic:                              96.18
Date:                Sun, 21 Jan 2024   Prob (F-statistic):                    3.31e-72
Time:                        13:22:31   Log-Likelihood:                         -11356.
No. Observations:                1379   AIC:                                  2.272e+04
Df Residuals:                    1375   BIC:                                  2.274e+04
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

In [46]:
df = pd.read_csv("DataWithoutPairs_USDRUB.csv")
df = df[df["Time"] > 3600]
df = df[df["Time"] < 30000]
df = df.reset_index()
df_OLS_anal(df)

                                 OLS Regression Results                                
Dep. Variable:                    SUM   R-squared (uncentered):                   0.030
Model:                            OLS   Adj. R-squared (uncentered):              0.029
Method:                 Least Squares   F-statistic:                              47.30
Date:                Sun, 21 Jan 2024   Prob (F-statistic):                    3.21e-39
Time:                        12:20:39   Log-Likelihood:                         -56672.
No. Observations:                6109   AIC:                                  1.134e+05
Df Residuals:                    6105   BIC:                                  1.134e+05
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

In [47]:
df = pd.read_csv("DataWithoutPairs_USDRUB.csv")
df = df[df["AskAfter"] - df["AskBefore"] > 0]
df = df[df["AskAfter"] - df["AskBefore"] < 0.03]
print(len(df["AskAfter"] - df["AskBefore"]))
df_OLS_anal(df)

3330
                                 OLS Regression Results                                
Dep. Variable:                    SUM   R-squared (uncentered):                   0.194
Model:                            OLS   Adj. R-squared (uncentered):              0.193
Method:                 Least Squares   F-statistic:                              200.4
Date:                Sun, 21 Jan 2024   Prob (F-statistic):                   3.70e-154
Time:                        12:20:50   Log-Likelihood:                         -27271.
No. Observations:                3327   AIC:                                  5.455e+04
Df Residuals:                    3323   BIC:                                  5.457e+04
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------